In [ ]:
import os, sys
current_path = os.getcwd()
current_path, tail = os.path.split(current_path)
folder_path= os.path.join(current_path, "libraries/")
file_path_ingest = folder_path + "Ingest.py"
file_path_calculations = folder_path + "Calculations.py"
%run $file_path_ingest
%run $file_path_calculations


In [ ]:
dir = r"D:\LiDAR\01_Projects\03_SRL\PI&T\Test Campaign\2024.07.08_SMX_VP02_VP04\TC13 Stop Sign Target Sep\2024-07-08_13-41-31.154_VPR002_Target1x1_GT25.04\RRL0283W"


In [ ]:
sensor_type = "SRL" 

if os.path.exists(dir) != True :logger.warning(f"Directory not found!, Is the drive connected?")
else:
    sensor_viper = sensor(sensor_type, dir, debug=True)


DATA INGEST

In [ ]:
sensor_viper.data_read(multiple_files=False)
sensor_viper.row_col_fix()
sensor_viper.data_validation()

----------

------------------------ SUMMARY BY EACH FRAME -----------------

In [ ]:
# data_valid=sensor_viper.df_valid_returns.groupby('frame_idx').describe()
# data_valid

----- Mean Amount of points per frame ----

In [ ]:
# sensor.df_valid_returns.groupby('frame_idx')['x'].count().mean()


---------------------------

In [ ]:
# sensor.df_valid_returns
# # df_test = sensor.df_valid_returns[(sensor.df_valid_returns['ring'] >= 125) & (sensor.df_valid_returns['ring'] <= 140)].copy()
# # df_test
# #px.scatter(sensor.df_valid_returns,x='y',y='z',color='SNR',animation_frame='frame_idx')
# #px.scatter_3d(sensor.df_single_frame,x='x',y='y',z='z',color='Processed Intensity')

In [ ]:
# df_test= sensor.df_valid_returns[(sensor.df_valid_returns['azimuth_angle'] >= 130) & (sensor.df_valid_returns['azimuth_angle'] <= 230)]
# px.scatter(df_test,x='azimuth_angle',y='elevation_angle',color='intensity')

## HESSAI PLOTTING BY AZIMUTH AND ELEVATION
#px.scatter(sensor.df_valid_returns,x='azimuth_angle',y='elevation_angle',color='intensity',animation_frame='frame_idx')

In [ ]:
## HESSAI PLOTTING BY COL AND ROW
#px.scatter(sensor.df_valid_returns,x='col',y='row',color='intensity',animation_frame='frame_idx')

In [ ]:
sensor_viper.df_single_frame = sensor_viper.df_single_frame[sensor_viper.df_single_frame['return_idx'] == 0] 

TARGET SELECTION

In [ ]:
target_selection(sensor_viper)
data_parting(sensor_viper)

In [ ]:
df_return_valid_count, pfa = pfa_calculation(sensor_viper)
pfa

In [ ]:
df_return_precision, average_std = range_precision(sensor_viper)
average_std

In [ ]:
# df_all_target=sensor.df_all_frames_target
# df_all_target.groupby('return_idx')['range'].mean()
# df_all_target.groupby('return_idx')['range'].quantile(q=0.5)
#planets.groupby('year')['orbital_period'].median()

In [ ]:
#df_return0=sensor.df_all_frames_target[sensor.df_all_frames_target['return_idx']==0].copy()
#sensor.df_valid_returns[sensor.df_valid_returns['frame_idx']==5].copy()
#df_return0
# sensor.df_selection

In [ ]:
# Q1=sensor.df_all_frames_target['range'].quantile(q=0.25)
# Q3=sensor.df_all_frames_target['range'].quantile(q=0.75)
# IQR=Q3-Q1
# High_Outlier= Q3+1.5*IQR
# Lower_Outlier= Q1-1.5*IQR
# print(High_Outlier)
# print(Lower_Outlier)
# df_all_frames_target_valid=sensor.df_all_frames_target[(sensor.df_all_frames_target['range']<=High_Outlier) & (sensor.df_all_frames_target['range']>=Lower_Outlier)]
# df_all_frames_target_valid.describe()

In [ ]:
# df_all_target=sensor.df_all_frames_target
# df_all_target.groupby('return_idx')['range'].min()


In [ ]:
pd_points = pd_calculation(sensor_viper)
range_readout = sensor_viper.df_all_frames_target['range'].mean()
ground_truth = 25.06
range_accuracy= ((ground_truth - sensor_viper.df_all_frames_target['range'].mean())**2)**0.5
print('Range Accuracy:', range_accuracy)

In [ ]:
TargHeight, TargWidth, TargPerimeter, TargArea= target_size(sensor_viper)
gt_width = .60
print("Target is about ",TargHeight,"m"" by", TargWidth," m", "Approximate Perimeter:",TargPerimeter, "Approximate Area:",TargArea)

In [ ]:
df_summary=pd.DataFrame({
                                    'Target Type':['Stop Sign 24"'],
                                    'Test Case':['TC_13'],
                                    'Ground Truth':[ground_truth],
                                    'Range Readout':[range_readout],
                                    'Range Accuracy':[range_accuracy],
                                    'Range Precision':[average_std],
                                    '90% PD Points':[pd_points],
                                    'GT Target Width':[gt_width],
                                    'Target Width':[TargWidth],
                                    'Error':[gt_width-TargWidth]
                                    })        
df_summary

In [ ]:
# directory_general = r"D:\LiDAR\01_Projects\03_SRL\PI&T\Test Campaign\2024.07.08_SMX_VP02_VP04\TC17 Kneeling Child\2024-07-08_14-19-28.052_VPR002_KneelingChild\RRL0283W"
df_summary.to_csv(f"{dir}/_SMX_StopSign10pVPR002_Small_{get_current_time_date()}.csv", sep=',')
